<a href="https://colab.research.google.com/github/ykitaguchi77/GravCont_classification_colab/blob/master/Eye_crop_bilateral_1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#顔の画像から目を検出して切り抜くスクリプト
・Haarcascade_eyeを使用<br>
・目が検出できないものはskipする<br>
・横幅を1/4倍、縦幅を上下に1/4倍追加して画像を切り取る（目の全幅、眉毛が含まれるように）

・切り取った画像を500px*500pxにresizeする<br>
・フォルダ内の画像を一括変換して別フォルダに保存

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import shutil
import copy
import pandas as pd
import csv
from random import randint
from time import sleep
import numpy as np
import sys


import glob
import random
from PIL import Image
%matplotlib inline

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2

plt.ion()   # interactive mode

#Google colabをマウント

In [64]:
'''
・dlibを用いて目を切り抜く
・横幅を2倍、縦幅を上に1倍追加/下に0.5倍追加して画像を切り取る（目の全幅、眉毛が含まれるように）
'''

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#残り時間確認
!cat /proc/uptime | awk '{printf("残り時間 : %.2f", 12-$1/60/60)}'


残り時間 : 10.64

In [7]:
#元画像
in_path = '/content/drive/MyDrive/Deep_learning/Face_Images/IMG_3110.JPG'

#保存先フォルダ
out_path = '/content/drive/My Drive/Deep_learning/eyelid_images/IMG_3110_eye.JPG'


#Haarcascadeを指定

In [8]:
# カスケードファイルのパス
eye_cascade_path = '/content/drive/My Drive/Deep_learning/haarcascade_eye.xml'

# カスケード分類器の特徴量取得
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

In [9]:
#アスペクト比を維持したまま横を400pixelに縮小する
def scale_to_width(img, width):
    scale = width / img.shape[1]
    return cv2.resize(img, dsize=None, fx=scale, fy=scale)

#図を表示する
def show_image(img):
    #img = cv2.imread(out_path)
    dst = scale_to_width(img, 100)
    cv2_imshow(dst)

def my_round(val, digit=0):
    p = 10 ** digit
    return int((val * p * 2 + 1) // 2 / p)

In [11]:
def crop_bilateral(in_path, size):
    img_resized_list,side_list = [],[]

    img = cv2.imread(in_path) 
    img2 = img.copy()

    # 画像グレースケール化
    grayscale_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    #300pix以上のもので目に見えるものを抽出
    eye_list = eye_cascade.detectMultiScale(grayscale_img, minSize=(300, 300))
    print('image path = ',in_path)

    # 眼検出判定
    if len(eye_list) >= 1:
        print('目が' + str(len(eye_list)) +'個検出されました')
        pass
    else:
        print("no eye detected")
        sys.exit()

    print(eye_list)

    #画像の切り抜きと保存（連番にする）
    if eye_list>= 1: 
        for (ex, ey, ew, eh) in eye_list:
            print("[ex,ey] = %d,%d [ew,eh] = %d,%d" %(ex, ey, ew, eh))
            
            try:
              cv2.rectangle(img2, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)

              #切り抜き範囲が元画像をはみ出る場合は黒画像で埋める
              top = max(0, ey-int(eh/4))
              bottom = min(grayscale_img.shape[0], int(ey + eh*5/4))
              left = max(0,int(ex-int(ew/4)))
              right = min(grayscale_img.shape[1], int(ex + ew*5/4))

              #print(f"top:{top}, bottom:{bottom}, left:{left}, right:{right}")

              img_cropped = img[top: bottom,left:right]
              height, width = img_cropped.shape[:2]


              #print(f"height:{height}, width:{width}")
              if width==height:
                  img_background =img_cropped
              elif width>height:
                  img_background = np.zeros((width,width,3), dtype = np.uint8)
                  img_background[0:width, my_round((width-height)/2) :int(((width+height)/2)+height)]=img_cropped
              elif width<height:
                  img_background = np.zeros((height,height,3), dtype = np.uint8)
                  img_background[0:height, my_round((height-width)/2) :int(((height-width)/2)+width)]=img_cropped
              
              img_resized = cv2.resize(img_background,(size,size)) #1辺を指定したpixにresize 

              #show_image(img_resized)

            except:
              print('crop error')

            if ex <= grayscale_img.shape[1]/2:
                side = 0 #600ピクセル目より左にあるのは右眼
            else:
                side = 1 #600ピクセル目より右にあるのは左眼

            try:
                img_resized_list.append(img_resized)
                side_list.append(side)
            except:
                pass

    return(img_resized_list, side_list)


In [ ]:
img_resized_list, side_list = crop_bilateral(in_path, 500)

#**ここからがメイン**

In [12]:
orig_folder = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset" #olympiaのデータセット
path_list = glob.glob(orig_folder+"/*")
path = path_list[0]
path


'/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/19.JPG'

In [13]:
orig_folder = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset" #olympiaのデータセット
dst_folder = "/content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset_224px_uni_periocular"


#処理時間の計測
start = time.time()

#もしdst_folderがあれば削除して新しく作り直す
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)
os.makedirs(dst_folder)

path_list = glob.glob(orig_folder+"/*")

for path in path_list:
    img_resized_list, side_list = crop_bilateral(path, 224) #両眼抜き出して224pxで保存（crop時にエラーが出るものは削除される）
    for img, side in zip(img_resized_list, side_list):
        if side == 0:
            sidestr = "R"
        elif side == 1:
            sidestr = "L"
        cv2.imwrite(dst_folder +"/"+ os.path.basename(path).split(".")[0]+"_"+sidestr+".JPG", img)

print('Process done!!')
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")


image path =  /content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/19.JPG
目が2個検出されました
[[ 177  627  696  696]
 [1584  618  708  708]]
[ex,ey] = 177,627 [ew,eh] = 696,696
[ex,ey] = 1584,618 [ew,eh] = 708,708
image path =  /content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/20.JPG
目が2個検出されました
[[1570  568  815  815]
 [ 152  561  785  785]]
[ex,ey] = 1570,568 [ew,eh] = 815,815
[ex,ey] = 152,561 [ew,eh] = 785,785
image path =  /content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/21.JPG
目が2個検出されました
[[1636  610  757  757]
 [ 219  627  723  723]]
[ex,ey] = 1636,610 [ew,eh] = 757,757
[ex,ey] = 219,627 [ew,eh] = 723,723
image path =  /content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/22.JPG
目が2個検出されました
[[ 135  463  888  888]
 [1624  494  858  858]]
[ex,ey] = 135,463 [ew,eh] = 888,888
crop error
[ex,ey] = 1624,494 [ew,eh] = 858,858
image path =  /content/drive/MyDrive/Deep_learning/Olympia_dataset/dataset/23.JPG
目が2個検出されました
[[ 175  499  695  695]
 [1676  519  772

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
